In [1]:
from seidr import SEIDR
from data_assimilation import OUTPUT_DIRECTORY_PATH, assimilate, apply_noise

In [2]:
import pandas as pd
import numpy as np

In [30]:
surogate = SEIDR(name="surogate",
                beta_values = [0.76,0.51, 0.26],
                gamma= 0.11,
                alpha=0.011,
                delta = 0.011
    ) 
surogate.calculate()
surogate.save_results()
surogate.plot()

In [31]:

baseline = pd.read_csv(f'{OUTPUT_DIRECTORY_PATH}/baseline.csv')
surogate = pd.read_csv(f'{OUTPUT_DIRECTORY_PATH}/surogate.csv')


In [36]:
def compare_results(SEIDR_real, SEIDR_pred):
    distance = 0.0
    for val_real, val_pred in zip(SEIDR_real, SEIDR_pred):
        distance+=np.linalg.norm(val_real-val_pred)/val_real.shape[0]
    return distance

In [37]:
def compare_var_matrices(baseline, surogate, baseline_vars, surogate_vars, deviation):
    values = {}
    noisy_baseline = apply_noise(baseline, 50)
    for baseline_var in baseline_vars:
        for surogate_var in surogate_vars:
            SEIDR_pred = assimilate(noisy_baseline, surogate, baseline_var, surogate_var)
            values[(baseline_var, surogate_var)] = compare_results(tuple(baseline[column].to_numpy() for column in baseline.columns)[1::], SEIDR_pred)
    return values

In [38]:
comparisons = compare_var_matrices(baseline, surogate, [0.5, 1,2, 4], [1,10,100,500, 1000], 100)

In [39]:
ser = pd.Series(list(comparisons.values()),
                  index=pd.MultiIndex.from_tuples(comparisons.keys()))
df = ser.unstack().fillna(0)
df

,1,10,100,500,1000
0.5,13.477261,11.802378,12.203853,12.251665,12.257789
1.0,16.240875,11.608778,12.147008,12.239516,12.251665
2.0,19.629305,11.747083,12.042854,12.215610,12.239516
4.0,22.724474,12.843426,11.871048,12.169359,12.215610


In [48]:
sample_nums = [400*i for i in range(1,21)]

In [49]:
results = []
for sample_num in sample_nums:
    sample_frequency = 400/sample_num
    baseline = SEIDR(f"baseline{sample_num}",sampling_frequency=sample_frequency)
    surogate = SEIDR(f"surogate{sample_num}",
                beta_values = [0.76,0.51, 0.26],
                gamma= 0.11,
                alpha=0.011,
                delta = 0.011,
                sampling_frequency=sample_frequency
    )
    baseline.calculate()
    surogate.calculate()

    baseline = baseline.to_pandas()
    surogate = surogate.to_pandas()

    noisy_baseline = apply_noise(baseline, 50)
    SEIDR_pred = assimilate(noisy_baseline, surogate, 1, 10)
    results.append(compare_results(tuple(baseline[column].to_numpy() for column in baseline.columns)[1::], SEIDR_pred))
    
results

[441.9887034439605,
 312.4736968394757,
 255.27145773209338,
 221.06802000233145,
 197.66964104479547,
 180.51416889818495,
 167.09437598342876,
 156.29843129289324,
 147.30175462163749,
 139.77923063502107,
 133.27508337095105,
 127.6097616056386,
 122.60859223230075,
 118.13649052770083,
 114.12330454306276,
 110.5237485639797,
 107.21484842822491,
 104.16957012541631,
 101.43660777929473,
 98.82281712532998]

In [ ]:
import matplotlib.pyplot as plt

t = list(range(1,21))


fig, ax = plt.subplots()
ax.plot(t, results)

ax.set(xlabel='time (s)', ylabel='voltage (mV)',
       title='About as simple as it gets, folks')
ax.grid()